In [ ]:
import tensorflow as tf
import cv2
from PIL import Image
import numpy as np
from skimage import transform
from collections import Counter
import os
import time

IMG_HEIGHT = 160
IMG_WIDTH = 160

buf = []
BUF_SIZE = 100  
THRESHOLD = 0.8  

root = "."
output_dir = './output_dir/'

# convert frame to image which is used for prediction.
def convert(np_image):
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (IMG_WIDTH, IMG_HEIGHT, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

# maintain a buffer for recent 100 frames
# if frequency of 0 in this buffer is larger than 0.8, return true  
def is_accident_detected(cur):  
    cur = cur.tolist()
    if len(buf) < BUF_SIZE:  
        buf.append(cur)
        return False
    buf.pop(0)  
    buf.append(cur)  
    countDict = Counter(buf)
    freqNum = countDict[0]  
    print('freqNum: ' + str(freqNum))
    if float(freqNum) / BUF_SIZE > THRESHOLD:
        return True
    else:
        return False

# load model
model = tf.keras.models.load_model('./model')

# load input video of CCTV
cap = cv2.VideoCapture('./fire.mp4')

cnt = -1
while(cap.isOpened()):
    ret, frame = cap.read()
    cnt += 1       
    img = convert(frame)
    print()
    print('now frame: ' + str(cnt))
    now_class = model.predict_classes(img)  # predict and yield output as class (0 or 1)
    now_class = np.squeeze(now_class)  # remove needless dimension
    if is_accident_detected(now_class):  # if accident is detected, print a frame at point of detection
        print('accident detected !!!' + ' in frame' + str(cnt))
        break

    cv2.imwrite(output_dir + str(cnt) + '.jpg', frame)  # write images in output directory

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()